In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
data_path = '/home/michael/nr/project/stat/ClimateFutures/RenewableEnergy/Statkraft_Brasil/Data/'
torch_path = '/home/michael/nr/project/stat/ClimateFutures/RenewableEnergy/Statkraft_Brasil/Michael_pytorch/'

Load csv files with inflow, precipitation, and temperature data aggregated to months and catchments

In [3]:
df_era5 = pd.read_csv(data_path+'Clean/era5_all.csv', encoding ='ISO-8859-1')
df_chirps = pd.read_csv(data_path+'Clean/Precipitation_CHIRPS_update.csv', encoding ='ISO-8859-1', index_col=0)
df_inflow = pd.read_csv(data_path+'Clean/inflow.csv', encoding ='ISO-8859-1')

In [11]:
catchment_names = np.intersect1d(np.intersect1d(np.unique(df_era5['catchid']), [f'catch{id}' for id in df_chirps.index]), np.unique(df_inflow['catchid']))
nbs = len(catchment_names)

months = [*range(1,13)]
years = [*range(1981,2021)]
nyrs = len(years)

In [10]:
df_catchment_info = pd.read_csv(data_path+'Clean/catchinfo.csv', encoding ='ISO-8859-1', index_col='catchid').reindex(catchment_names)

Reformat data and calculate lagged precipitation and temperature variables

In [20]:
inflow = np.full((nbs,12,nyrs), np.nan, dtype=float)
temp_lagged = np.full((nbs,12,nyrs,6), np.nan, dtype=float)
precip_lagged = np.full((nbs,12,nyrs,6), np.nan, dtype=float)

for ibs in range(nbs):
    id_catchment = catchment_names[ibs]
    temp_catchment = df_era5.loc[(df_era5['catchid']==id_catchment),['year','month','temp']].rename(columns={'temp':'temp_lg0'})
    prcp_catchment = df_chirps.loc[int(id_catchment[5:])].to_frame().rename(columns={int(id_catchment[5:]):'precip_lg0'})
    prcp_catchment.index = pd.date_range('1981-01-01', periods=len(prcp_catchment), freq='MS')
    prcp_catchment.insert(0, 'month', prcp_catchment.index.month)
    prcp_catchment.insert(1, 'year', prcp_catchment.index.year)
    for lg in range(1,6):
        temp_catchment.insert(3, f'temp_lg{lg}', temp_catchment['temp_lg0'].shift(lg))
        prcp_catchment.insert(3, f'precip_lg{lg}', prcp_catchment['precip_lg0'].shift(lg))
    for imt in range(12):
        month = months[imt]
        inflow[ibs,imt,:] = df_inflow.loc[(df_inflow['catchid']==id_catchment) & (df_inflow['month']==month)].set_index('year').loc[years,'inflow']
        if np.unique(inflow[ibs,imt,:]).size < 10:
            inflow[ibs,imt,:] = np.nan                 # discard catchment-month combinations with too many identical inflow values (this is suspicious)
            continue        
        for lg in range(6):
            temp_lagged[ibs,imt,:,lg] = temp_catchment.loc[temp_catchment['month']==int(month)].set_index('year').loc[years,f'temp_lg{lg}']
            precip_lagged[ibs,imt,:,lg] = prcp_catchment.loc[prcp_catchment['month']==int(month)].set_index('year').loc[years,f'precip_lg{lg}']


Standardize variables and calculate correlations

In [ ]:
inflow_mean = np.mean(inflow, axis=2)
inflow_std = np.std(inflow, axis=2)
y = (inflow-inflow_mean[:,:,None]) / inflow_std[:,:,None]

temp_lagged_mean = np.nanmean(temp_lagged, axis=2)
temp_lagged_std = np.nanstd(temp_lagged, axis=2)
X1 = np.nan_to_num((temp_lagged - temp_lagged_mean[:,:,None,:]) / temp_lagged_std[:,:,None,:], 0.0)

precip_lagged_mean = np.nanmean(precip_lagged, axis=2)
precip_lagged_std = np.nanstd(precip_lagged, axis=2)
X2 = np.nan_to_num((precip_lagged - precip_lagged_mean[:,:,None,:]) / precip_lagged_std[:,:,None,:], 0.0)

corr_coef_temp = np.mean(y[:,:,:,None]*X1, axis=2)
corr_coef_precip = np.mean(y[:,:,:,None]*X2, axis=2)


Depict correlation coefficients of precipitation and streamflow in a map  for different months

In [ ]:
lon_bounds = [-70., -34.]   # Brazil
lat_bounds = [-31.,  3.]    # Brazil

month_name = ['January','February','March','April','May','June','July','August','September','October','November','December']

fig, axs = plt.subplots(nrows=4, ncols=6, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(18,12.5))
for im in [0,3,6,9]:
    for ilg in range(6):
        axs[im//3,ilg].set_extent(lon_bounds+lat_bounds, crs=ccrs.PlateCarree())
        axs[im//3,ilg].add_feature(cfeature.LAND, color="lightgrey", alpha=0.75)
        axs[im//3,ilg].add_feature(cfeature.COASTLINE)
        axs[im//3,ilg].add_feature(cfeature.BORDERS, linestyle='-', alpha=.5)
        axs[im//3,ilg].add_feature(cfeature.LAKES, alpha=0.95)
        axs[im//3,ilg].add_feature(cfeature.RIVERS)
        axs[im//3,ilg].set_title(f'{month_name[im][:3]} streamflow - {month_name[(im-ilg)%12][:3]} precip.', fontsize=14)
        cmap, vmin, vmax = 'BrBG', -1.0, 1.0  # 'PuOr'
        cscatter = axs[im//3,ilg].scatter(x=df_catchment_info['Lon'], y=df_catchment_info['Lat'], c=corr_coef_precip[:,im,ilg], cmap=cmap, s=10, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)

fig.subplots_adjust(bottom=0.07, top=0.98, left=0.01, right=0.99, wspace=0.01, hspace=0.11)
cbar_ax = fig.add_axes([0.2, 0.025, 0.6, 0.025])
cbar = plt.colorbar(cscatter, cax=cbar_ax, orientation='horizontal', extend='both')
ticklabs = cbar.ax.get_xticklabels()
cbar.ax.set_xticklabels(ticklabs, fontsize=14)

Same for temperature and streamflow

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=6, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(18,12.5))
for im in [0,3,6,9]:
    for ilg in range(6):
        axs[im//3,ilg].set_extent(lon_bounds+lat_bounds, crs=ccrs.PlateCarree())
        axs[im//3,ilg].add_feature(cfeature.LAND, color="lightgrey", alpha=0.75)
        axs[im//3,ilg].add_feature(cfeature.COASTLINE)
        axs[im//3,ilg].add_feature(cfeature.BORDERS, linestyle='-', alpha=.5)
        axs[im//3,ilg].add_feature(cfeature.LAKES, alpha=0.95)
        axs[im//3,ilg].add_feature(cfeature.RIVERS)
        axs[im//3,ilg].set_title(f'{month_name[im][:3]} streamflow - {month_name[(im-ilg)%12][:3]} temp.', fontsize=14)
        cmap, vmin, vmax = 'BrBG', -1.0, 1.0  # 'PuOr'
        cscatter = axs[im//3,ilg].scatter(x=df_catchment_info['Lon'], y=df_catchment_info['Lat'], c=corr_coef_temp[:,im,ilg], cmap=cmap, s=10, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)

fig.subplots_adjust(bottom=0.07, top=0.98, left=0.01, right=0.99, wspace=0.01, hspace=0.11)
cbar_ax = fig.add_axes([0.2, 0.025, 0.6, 0.025])
cbar = plt.colorbar(cscatter, cax=cbar_ax, orientation='horizontal', extend='both')
ticklabs = cbar.ax.get_xticklabels()
cbar.ax.set_xticklabels(ticklabs, fontsize=14)